In [1]:
import os, sys

colab = True
# #### COLAB ONLY CELL
if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    # drive.mount('/content/gdrive',force_remount=True)
    os.chdir("/content/gdrive/MyDrive/COMEXT")
    !pip install -q -r world-trade-thesis/requirements.txt
    !pwd
    sys.path.append('/content/gdrive/MyDrive/COMEXT/world-trade-thesis/src')
else:
    os.chdir("H:/My Drive/COMEXT")
    !pwd
    sys.path.append('H:/My Drive/COMEXT/world-trade-thesis/src')

# import random, math, os, pickle, sys, powerlaw, hashlib #,eurostat
# import pandas as pd
# import numpy as np
# import networkx as nx
# from tqdm.notebook import tqdm, trange
# from networkx.readwrite import json_graph
# from networkx.classes.function import degree
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
# from scipy.optimize import curve_fit
# from sklearn.preprocessing import MinMaxScaler

import graph_analysis as g
if colab:
    import tqdm.notebook as tqdm
# exec(open("world-trade-thesis/src/graph_analysis.py","r").read())

# df_pop, eu_iso = g.load_population_df()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/COMEXT
Functions loaded!


In [ ]:
import pickle
import pandas as pd
from tqdm import tqdm
import networkx as nx
import graph_analysis as g
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

print("Running on whole time and merging to big df...")
metrics_full = {}
df_pop, eu_iso = g.load_population_df()
big_df_col = ["year","month","prod","country"] +  ['edges','nodes','size','degree',
    'degree_perc',
    'out_degree',
    'out_degree_perc',
    'in_degree',
    'in_degree_perc',
    'density',
    'density_adj',
    'clustering',
    'page_rank',
    'vulnerability',
    'in_weight_degree',
    'out_weight_degree',
    'degree_gamma',
    'degree_xmin',
    'in_degree_gamma',
    'in_degree_xmin',
    'out_degree_gamma',
    'out_degree_xmin',
    'in_weight_degree_gamma',
    'in_weight_degree_xmin',
    'out_weight_degree_gamma',
    'out_weight_degree_xmin',
    'out_weight_abs',
    'in_weight_abs']

big_df = pd.DataFrame()
issues = []
for y in tqdm([y for y in range(2001,2021)]):
    df = g.load_filtered_data("complete", save=True, force_reload=True, verbose=False, sorted=False, columns=g.full_columns, types=g.full_types, group_by_prod_code=True, n_digits=2,
                                years=[y], months=None, product="all", flow="all", declarant_iso="all", partner_iso="all", trade_type="all")
    n4g = g.get_world_countries(df_pop, eu_iso, year=str(y))
    for prod in tqdm(df.PRODUCT_CPA2_1.unique(),leave=False):
        df4g = df[df.PRODUCT_CPA2_1 == prod]
        t4g = g.extract_table_for_graph(df4g, y=str(y), flow="all", scale_by='population', pop_df=df_pop)
        if t4g.shape[0] < 10:
            issues.append((y,prod,t4g.shape[0]))
        coord, metrics, G = g.makeGraph(t4g, tab_nodes=None, weight_flag=True, criterio="VALUE_IN_EUROS", compute_layout=False, colab=colab)
        nx.write_gexf(G,f"data-samples/graphs/complete/complete_y{y}_p{prod}.gexf")
        x = metrics.reset_index().to_dict()
        x["year"] = str(y)
        x["month"] = "-"
        x["prod"] = prod
        big_df = pd.concat([big_df,pd.DataFrame(x,columns=big_df_col)],axis=0)
        metrics_full[f"{y}_cpa_{prod}"] = metrics
    #     break
    # break

version = 3

with open(f"./data-samples/metrics/issues_complete_{version}.txt","w") as f:
    f.write(str(issues))
pickle.dump(metrics_full,open(f"./data-samples/metrics/metrics_complete_V_{version}.pickle","wb"))
big_df.to_parquet(f"./data-samples/metrics/metrics_complete_V_{version}.parquet")
